In [1]:
import yfinance as yf 
from yahoofinancials import YahooFinancials

from datetime import date, timedelta
import requests
import numpy as np
import pandas as pd
import warnings
import pandas_datareader as web
import seaborn as sns 
from matplotlib.pyplot import show
from scipy import stats
# import yahooquery
warnings.filterwarnings('ignore')
today = str(date.today())

c:\Users\jinfe\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\jinfe\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


     ---------------------------------------- 52.7/52.7 kB 1.4 MB/s eta 0:00:00
     ---------------------------------------- 3.9/3.9 MB 12.4 MB/s eta 0:00:00
     --------------------------------------- 11.6/11.6 MB 25.2 MB/s eta 0:00:00
     ---------------------------------------- 78.3/78.3 kB ? eta 0:00:00
     -------------------------------------- 147.9/147.9 kB 9.2 MB/s eta 0:00:00
     ------------------------------------- 345.4/345.4 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.1
    Uninstalling tqdm-4.64.1:
      Successfully uninstalled tqdm-4.64.1
  Attempting uninstall: lxml
    Found existing installation: lxml 4.9.1
    Uninstalling lxml-4.9.1:
      Successfully uninstalled lxml-4.9.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.1
    Uninstalling beautifulsoup4-4.11.1:
      Successfully uninstalled beautifulsoup4-4.11.1
  Attempting uninstall: pandas
    Found existing 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.5.0 requires numpy<1.24,>=1.16.0, but you have numpy 1.24.4 which is incompatible.
ydata-profiling 4.5.0 requires pandas!=1.4.0,<2.1,>1.1, but you have pandas 2.2.2 which is incompatible.


In [21]:
def calculate_growth(df, column, transpose = True):
    if transpose == True:
        sort_df = df.T.sort_index(ascending=False)[[column]]
    else:
        sort_df = df.sort_index(ascending=False)[[column]]
    growth_col = column + ' growth'
    sort_df[growth_col] = sort_df[column] / sort_df[column].shift(-1)
    return sort_df, sort_df[growth_col].iloc[0]

def get_kpi(stock_ticker):
    company_df = yf.Ticker(stock_ticker)

    income_stm = company_df.income_stmt[company_df.income_stmt.index.isin(['Total Operating Income As Reported','Net Income','Cost Of Revenue', 'Total Revenue', 'Operating Revenue', 'Gross Profit'])]
    balance_stm = company_df.balance_sheet[company_df.balance_sheet.index.isin(['Total Assets', 'Cash Cash Equivalents And Short Term Investments','Current Liabilities'])]
    cf_stm = company_df.cashflow[company_df.cashflow.index.isin(['Free Cash Flow','Operating Gains Losses'])]

    # statement = income_stm.append(balance_stm).append(cf_stm)
    statement = pd.concat([income_stm, balance_stm, cf_stm])
    curr_stm = statement[statement.columns.max()]

    # KPI
    # profitability
    profitability = {
        'revenue': curr_stm['Total Revenue'],
        'cost': curr_stm['Cost Of Revenue'],
        'operating_income': curr_stm['Total Operating Income As Reported'],
        'net_profit': curr_stm['Gross Profit'],
        'total_asset': curr_stm['Total Assets'],
        'gross_margin' : company_df.info['grossMargins'],
        'operating_margin' : company_df.info['operatingMargins'],  
        'net_margin' : company_df.info['profitMargins'],    
        'ROA' : company_df.info['returnOnAssets'],
        'ROE' : company_df.info['returnOnEquity'] 
    }
    profitability = pd.DataFrame.from_dict(profitability, orient='index', columns=['Value']).reset_index()


    # Financial health
    financial_health = {
        'cashflow_over_cost' : curr_stm['Free Cash Flow'] / curr_stm['Cost Of Revenue'],
        'current_ratio' : company_df.info['currentRatio'],
        'liability_asset_ratio' : curr_stm['Current Liabilities'] / curr_stm['Total Assets']
    } 
    financial_health = pd.DataFrame.from_dict(financial_health, orient='index', columns=['Value']).reset_index()


    # Growth 
    _, operating_income_growth = calculate_growth(statement, column = 'Total Operating Income As Reported', transpose = True)
    _, free_cashflow_growth = calculate_growth(statement, column = 'Free Cash Flow', transpose = True)
    growth = {
        'revenue_growth' : company_df.info['revenueGrowth'],
        'net_income_growth' : company_df.info['earningsGrowth'],
        'operating_income_growth': operating_income_growth,
        'free_cashflow_growth': free_cashflow_growth
    }
    growth = pd.DataFrame.from_dict(growth, orient='index', columns=['Value']).reset_index()

    # Operational efficiency 
    # sales_cost_over_revenue
    # inventory_days
    # all_kpi = profitability.append(financial_health).append(growth)
    all_kpi = pd.concat([profitability, financial_health,growth])
    output = {
        'statement' : statement,
        'profitability': profitability, 
        'financial_health': financial_health, 
        'growth': growth,
        'all_kpi': all_kpi
    }

    return output



def get_multiple_stocks_kpi(stock_list):
    """ 
    stock_list: list of stock ticker

    kpi to be collected: revenue, cost, operating income, operating margin, ROA, liability asset ratio, profit 
    """
    all_company_kpi = pd.DataFrame()
    for ticker in stock_list:
        company_kpi = get_kpi(stock_ticker = ticker)['all_kpi']
        company_kpi['Company'] = ticker
        # all_company_kpi = all_company_kpi.append(company_kpi)
        all_company_kpi = pd.concat([all_company_kpi, company_kpi])
    
    return all_company_kpi

def plot_with_diagonal(plot_df, x_col, y_col, hue_col, title=None, diagonal = True):
    plot = sns.scatterplot(data=plot_df, x=x_col, y=y_col, hue=hue_col)
    if diagonal == True:
        x0, x1 = plot.get_xlim()
        y0, y1 = plot.get_ylim()
        lims = [0, min(x1, y1)]
        plot.plot(lims, lims, '-r')
    plot.set_title(title)
    show()

In [ ]:
testdf = get_multiple_stocks_kpi(['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'AMZN', 'META', 'AVGO', 'TSLA', 'V'])
testdf = testdf.pivot(index='Company', columns='index', values='Value').reset_index()
testdf

In [29]:
company_kpi = get_kpi(stock_ticker = 'AAPL')['statement']
company_kpi = company_kpi.melt(ignore_index=False).reset_index()

In [31]:
import plotly.express as px

fig = px.line(company_kpi, x="variable", y="value", color='index')
fig.show()

In [52]:
company_kpi = yf.Ticker('AAPL').income_stmt['2023-09-30'].reset_index()
company_kpi.columns = ['index', 'value']
company_kpi['company'] = 'AAPL'
company_kpi[~company_kpi.index.isin(['Total Revenue', 'Operating Revenue'])]



,index,value,company
0,Tax Effect Of Unusual Items,0.0,AAPL
1,Tax Rate For Calcs,0.147,AAPL
2,Normalized EBITDA,129188000000.0,AAPL
3,Net Income From Continuing Operation Net Minor...,96995000000.0,AAPL
4,Reconciled Depreciation,11519000000.0,AAPL
5,Reconciled Cost Of Revenue,214137000000.0,AAPL
6,EBITDA,129188000000.0,AAPL
7,EBIT,117669000000.0,AAPL
8,Net Interest Income,-183000000.0,AAPL
9,Interest Expense,3933000000.0,AAPL


In [53]:
import plotly.express as px

fig = px.bar(company_kpi[~company_kpi.index.isin(['Total Revenue', 'Operating Revenue'])], x="company", y="value", color = 'index', title="total revenue")
fig.update_layout(barmode="stack")
fig.show()